In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os, sys
BASE = "/content/drive/MyDrive/hack_3"
os.chdir(BASE)
if BASE in sys.path:
    sys.path.remove(BASE)
sys.path.insert(0, BASE)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from config import PipelineConfig
import importlib, person_graph_builder, pipeline_demo
importlib.reload(person_graph_builder)
importlib.reload(pipeline_demo)

from pipeline_demo import run

cfg = PipelineConfig(
    device="cuda",
    max_articles=100000000,
    use_nli=True,
    use_risk=True,
    nli_threshold=0.82,
)

index = run(
    csv_path="sample.csv",
    out_path="person_index.json",
    cfg=cfg
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0
Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Saved: person_index.json
Persons: 713


In [6]:
from person_search import PersonIndexSearch

s = PersonIndexSearch.load("person_index.json")

# 1) статистика по базе
s.print_stats_global()
s.print_top_persons(top_k=30)

# 2) персональная карточка (все связанные сущности + риски + short relations)
s.print_person_card("Putin", top_n_each_type=15, min_support_articles=2, show_evidence=True)

# 3) только “смысловые связи” (works_for / appointed_to / met_with / related_to)
s.print_semantic_relations("Putin", min_nli=0.82, top_n=30)


GLOBAL STATS
persons_total: 713
neighbors_total: 3097
neighbors_by_type: {'person': 1157, 'organization': 636, 'location': 1304, 'unknown': 0}
nli_labels: {'works for': 301, 'met with': 167, 'was appointed to': 4}
TOP PERSONS BY NEIGHBORS (top_k=30)
01. Donald Tramp | total=182 (PER=47, ORG=39, LOC=96) | risk=LOW (0.21)
02. İlham Əliyev | total=65 (PER=17, ORG=10, LOC=38) | risk=LOW (0.20)
03. Volodimir Zelenski | total=60 (PER=19, ORG=11, LOC=30) | risk=LOW (0.20)
04. Vladimir Putin | total=35 (PER=10, ORG=4, LOC=21) | risk=LOW (0.20)
05. İlham Əliyevin | total=34 (PER=7, ORG=7, LOC=20) | risk=LOW (0.00)
06. Putin | total=33 (PER=12, ORG=4, LOC=17) | risk=LOW (0.20)
07. Emmanuel Makron | total=32 (PER=11, ORG=3, LOC=18) | risk=LOW (0.20)
08. Rəcəb Tayyib Ərdoğan | total=32 (PER=4, ORG=7, LOC=21) | risk=LOW (0.20)
09. Marko Rubio | total=28 (PER=12, ORG=6, LOC=10) | risk=LOW (0.20)
10. Hakan Fidan | total=27 (PER=7, ORG=9, LOC=11) | risk=LOW (0.00)
11. Makron | total=27 (PER=10, ORG=4,

In [7]:
s.print_person_card(
    "Vladimir Putin",
    top_risk_types=0,            # вывести все
    show_risk_types=True,
    show_risk_evidence=True
)


PERSON CARD: Vladimir Putin (key=vladimir putin, match=1.00)
RISK: LOW (0.20)
RISK TYPES: none
------------------------------------------------------------------------------------------
NEIGHBORS (min_support_articles=2):

[PERSON] top=10
- Putin | support=49 | score=20.94 | nli=met with (1.00)
  * evidence: “Rusiya Prezidenti Vladimir Putinlə onun üçün əlverişli olan istənilən vaxt görüşməyə hazıram”.
  * link: https://report.az/diger-olkeler/putinle-gorusmeye-hazir-olan-tramp-kiyeve-silah-tedarukunu-dayandirmagi-dusunur
- Tramp | support=65 | score=14.83 | nli=met with (0.93)
  * evidence: ABŞ-nin yeni seçilmiş Prezidenti Donald Tramp Rusiya lideri Vladimir Putinlə danışıqlar aparmaq istədiyini bildirib.
  * link: https://report.az/diger-olkeler/tramp-yaxin-alti-ay-erzinde-putinle-danisiqlar-aparmaq-istediyini-bildirib
- Volodimir Zelenski | support=20 | score=13.96 | nli=met with (0.99)
  * evidence: “Mən, yəqin ki, gələn həftə Prezident Zelenski ilə görüşəcəyəm və yəqin ki, Rusiya 